In [88]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import csv
from parsel import Selector
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pickle
import re
import pandas as pd
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from random import randint, shuffle
from datetime import date
import dill as pickle
from collections import defaultdict
import datetime
import random

In [4]:
from dotenv import load_dotenv
import os 
import sys

src_dir = os.path.abspath(os.path.join(os.pardir, 'src'))
sys.path[0] = src_dir
import paths

load_dotenv(dotenv_path=os.path.join(paths.repository_root, '.env'))
user_id = os.getenv('USER_EMAIL')
user_pw = os.getenv('PASSWORD')

# Get linked in profiles from Google

In [5]:
location = 'Greater Chicago Area'
query =  'site:linkedin.com/in/ AND "interaction designer" AND "Chicago"'

In [6]:
def open_driver():
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.get("https://www.google.com/")
    return driver

In [100]:
driver = open_driver()


[WDM] - Looking for [chromedriver 85.0.4183.87 mac64] driver in cache 
[WDM] - File found in cache by path [/Users/alee/.wdm/drivers/chromedriver/85.0.4183.87/mac64/chromedriver]


In [6]:
def get_linkedin_profile_urls(driver, query, end=10, sleep_time=3):
    """
    get linkedin profile urls from google search
    input:
        driver: selenium driver
        query: query for searching starts with site:linkedin.com/in/ 
        end: define how many urls we want
    output:
        linkedin_profile_urls: list of linkedin profile urls in string
    """
    
    linkedin_profile_urls = []
    # find the search section of google
    search_input = driver.find_element_by_name('q')
    search_input.clear()
    # search query 
    search_input.send_keys(query)
    search_input.send_keys(Keys.RETURN)
    
    #result stats, get how many results that google estimates
    result_stats = driver.find_element_by_id("result-stats").text
    result_num = re.findall(r'About (\d+(?:,\d+)?) results', result_stats)[0]
    result_num = int(result_num.replace(',', ''))
    
    # iterate until the length of list meets our goal
    page_num = 1
    while len(linkedin_profile_urls) < end:
        try:
            time.sleep(3*sleep_time) # adding sleep time so that google doesn't reject us
            # get profiles under class='r'
            profiles = driver.find_elements_by_xpath('//*[@class="r"]/a[1]')
            linkedin_profile_urls.extend([p.get_attribute('href') for p in profiles])
            linkedin_profile_urls = list(set(linkedin_profile_urls))
            # if the list is larger than our goal, cut rest of it off
            if len(linkedin_profile_urls) > end:
                linkedin_profile_urls = linkedin_profile_urls[:end]
                break
            # scroll down to the end of page like humam
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight/4);")
            time.sleep(sleep_time)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
            time.sleep(sleep_time)
            driver.execute_script("window.scrollTo(0, 3*document.body.scrollHeight/4);")
            time.sleep(sleep_time)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(sleep_time)
            # go to the next page
            # keep track of page number so when google goes to captcha we can restart at the page we lost
            page_num += 1
            driver.find_element_by_id("pnnext").click()
        except:
            # if google thinks we are going to fast, sleep it off. usually we need to refresh the driver
            # need a better way 
            print(len(linkedin_profile_urls))
            time.sleep(10*sleep_time)
            print('restart')
            driver.get("https://www.google.com/")
            search_input = driver.find_element_by_name('q')
            search_input.clear()
            # search query 
            search_input.send_keys(query)
            search_input.send_keys(Keys.RETURN)
            driver.find_element_by_id("pnnext").click()
            driver.find_element_by_xpath('//a[@aria-label="Page {}"]'.format(page_num)).click()
            continue
    return linkedin_profile_urls, result_num

In [7]:
linkedin_profile_urls, result_num = get_linkedin_profile_urls(driver, query, 12)

In [8]:
linkedin_profile_urls

['https://www.linkedin.com/in/nada-dabbouseh',
 'https://www.linkedin.com/in/geefit',
 'https://www.linkedin.com/in/ansarileena',
 'https://www.linkedin.com/in/tyler-stober-4010974a',
 'https://www.linkedin.com/in/theresalundeen',
 'https://www.linkedin.com/in/mabburke',
 'https://www.linkedin.com/in/danifold',
 'https://www.linkedin.com/in/corey-swimmer',
 'https://www.linkedin.com/in/melissamanak',
 'https://www.linkedin.com/in/thejustinwu',
 'https://www.linkedin.com/in/jameszulkie',
 'https://www.linkedin.com/in/nickrathert']

# Search LinkedIn

In [8]:
with open('../data/linkedin_profiles_rui.pkl', 'rb') as fp:
    profile_urls = pickle.load(fp)

In [9]:
profile_urls

['https://www.linkedin.com/in/sammcmorran/',
 'https://www.linkedin.com/in/peihwa-lan-5416b664/',
 'https://www.linkedin.com/in/will-dai/',
 'https://www.linkedin.com/in/xiaominliu516/',
 'https://www.linkedin.com/in/faye-fei-zhao/',
 'https://www.linkedin.com/in/weiyiping-huang-76801047/',
 'https://www.linkedin.com/in/sean-fu-bab515176/',
 'https://www.linkedin.com/in/kaiitu/',
 'https://www.linkedin.com/in/wendy-wang-mertensmeyer-8386b018/',
 'https://www.linkedin.com/in/victoria66/',
 'https://www.linkedin.com/in/yeovanessa/',
 'https://www.linkedin.com/in/brandon-ly-b8236344/',
 'https://www.linkedin.com/in/violet-hill-05a7b0108/',
 'https://www.linkedin.com/in/ailun-s-5a8368117/',
 'https://www.linkedin.com/in/lin-zhang-62042870/',
 'https://www.linkedin.com/in/bryantmao/',
 'https://www.linkedin.com/in/yue-yuan-b3a853115/',
 'https://www.linkedin.com/in/rui-xu-5130a799/',
 'https://www.linkedin.com/in/lisa-xue-510a355b/',
 'https://www.linkedin.com/in/kikozang/',
 'https://www.l

In [10]:
def login_linkedin(driver, linkedin_id, linkedin_pw):
    driver.get('https://www.linkedin.com')
    driver.find_element_by_xpath('//a[text()="Sign in"]').click()
    time.sleep(5)
    username_input = driver.find_element_by_name('session_key')
    username_input.send_keys(linkedin_id)

    password_input = driver.find_element_by_name('session_password')
    password_input.send_keys(linkedin_pw)
    driver.find_element_by_xpath('//button[text()="Sign in"]').click()
    return driver

In [101]:
driver = login_linkedin(driver, user_id, user_pw)

In [75]:
profile = profile_urls[2]
driver.get(profile)

In [76]:

def get_basic_info(driver, profile):
    #linkedin url
    ln_url = profile
    #name 
    name = driver.find_element_by_xpath('//ul//li[@class="inline t-24 t-black t-normal break-words"]').text
    # current title
    title = driver.find_element_by_xpath('//h2[@class="mt1 t-18 t-black t-normal break-words"]').text
    # location
    location = driver.find_element_by_xpath('//ul//li[@class="t-16 t-black t-normal inline-block"]').text
    return name, title, location

In [81]:
def get_contacts(driver, profile, name):
    contact_data = []
    contact_link = driver.find_element_by_xpath('//a[@data-control-name="contact_see_more"]')
    contact_link.click()
    time.sleep(3)
    contacts = driver.find_elements_by_xpath("//a[contains(@class, 'pv-contact-info__contact-link')]")#.get_attribute('href')
    contacts = [i.get_attribute('href') for i in contacts]
    other_contacts = [i for i in contacts if 'linkedin' not in i and '@' not in i]
    emails = [i.replace('mailto:', '') for i in contacts if '@' in i]
    driver.find_element_by_xpath('//button[@aria-label="Dismiss"]').click()
    for contact in emails+other_contacts:
        contact_data.append([profile, name, contact])
    return contact_data

In [17]:

def get_start_end_date(duration):
    duration = duration.replace('Dates Employed\n','')
    start_date, end_date = duration.split(' – ')
    start_date = datetime.datetime.strptime(start_date, '%b %Y').date()
    if end_date == 'Present':
        end_date = datetime.datetime.today().date()
    else:
        end_date = datetime.datetime.strptime(end_date, '%b %Y').date()
    return start_date, end_date

In [64]:
def get_experience(driver, profile, name):
    experience_data = []
    exp_section = driver.find_element_by_xpath('//section[@id="experience-section"]')
    experiences = exp_section.find_elements_by_xpath('.//li[@class="pv-entity__position-group-pager pv-profile-section__list-item ember-view"]')
    for i, exp in enumerate(experiences):
        try: #when one only had one position
            company = exp.find_element_by_xpath('.//a[@data-control-name="background_details_company"]')
            job_title = company.find_element_by_tag_name('h3').text.replace('Company Name\n', '')
            company_name = company.find_element_by_tag_name('img').get_attribute('alt')
            duration = company.find_element_by_xpath('.//h4[contains(@class, "pv-entity__date-range")]').text
            start_date, end_date = get_start_end_date(duration)
            try:
                location = company.find_element_by_xpath('.//h4[contains(@class, "pv-entity__location")]').text.replace('Location\n', '')
            except:
                location = ''
            experience_data.append([profile, name, company_name, job_title, start_date, end_date])
        except: # when there were multiple positions
            company = exp.find_element_by_xpath('.//section[contains(@class,"pv-profile-section")]')
            company_name = company.find_element_by_tag_name('img').get_attribute('alt')
            roles = company.find_elements_by_xpath('.//li[contains(@class, "pv-entity__position")]')
            for r in roles:
                job_title = r.find_element_by_tag_name('h3').text.replace('Title\n', '')
                duration = company.find_element_by_xpath('.//h4[contains(@class, "pv-entity__date-range")]').text
                start_date, end_date = get_start_end_date(duration)
                experience_data.append([profile, name, company_name, job_title, start_date, end_date])

    return experience_data

In [120]:
def get_education(driver, profile, name):
    education_data = []
    education_section = driver.find_element_by_xpath('*//section[@id="education-section"]')
    educations = education_section.find_elements_by_xpath('.//li[contains(@class, "pv-education-entity")]')
    for ed in educations:
        school_name = ed.find_element_by_xpath('.//h3[contains(@class, pv-entity__school-name)]').text
        ed_info = [e.text for e in ed.find_elements_by_xpath('.//p[contains(@class, pv-entity)]')]
        degree = [i for i in ed_info if 'Degree Name' in i]
        if len(degree) > 0:
            degree = degree[0].replace('Degree Name\n', '')
        else:
            degree = ''
        fos = [i for i in ed_info if 'Field Of Study' in i]
        if len(fos) > 0:
            fos = fos[0].replace('Field Of Study\n', '')
        else:
            fos = ''
        date_range = [i for i in ed_info if 'Dates attended or expected graduation' in i]
        if len(date_range) > 0:
            date_range = date_range[0].replace('Dates attended or expected graduation\n', '')
            start_year, end_year = date_range.split(' – ')

        else:
            date_range = ''
            start_year = None
            end_year = None

        education_data.append([profile, name, school_name, degree, fos, start_year, end_year])
    return education_data

In [109]:
def get_language(driver):
    try:
        language = driver.find_element_by_xpath('.//div[@aria-labelledby="languages-title"]').text
    except:
        language = ''
    return language

In [108]:
get_language(driver).text

'English Korean'

In [121]:
basic_info_data_all = []
contacts_data_all = []
experience_data_all = []
education_data_all = []
language_data_all = []

In [122]:
for i, profile in enumerate(profile_urls):
    t = random.uniform(1.5,4.5)
    driver.get(profile)
    time.sleep(t)
    name, title, location = get_basic_info(driver, profile)
    print(i, name, '{:.2f}'.format(t))
    basic_info_data_all.append([profile, name, title, location])
    time.sleep(t)
    contact_data = get_contacts(driver, profile, name)
    contacts_data_all.extend(contact_data)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight/3);")
    time.sleep(2*t)
    experience_data = get_experience(driver, profile, name)
    experience_data_all.extend(experience_data)
    driver.execute_script("window.scrollTo(0, 2*document.body.scrollHeight/3);")
    time.sleep(t)
    try:
        education_data = get_education(driver, profile, name)
        education_data_all.extend(education_data)
    except: # no education section
        continue
    driver.execute_script("window.scrollTo(0, 3*document.body.scrollHeight/4);")
    time.sleep(t)
    language = get_language(driver)
    language_data_all.append([profile, name, language])
    time.sleep(5*t)

0 Sam McMorran 3.47
1 Peihwa Lan 4.34
2 Will Dai 1.71
3 Xiaomin Liu 2.21
4 Faye (Fei) Zhao 1.98
5 Weiyiping Huang 3.43
6 Sean Fu 3.07
7 Kaii T. 4.07
8 Wendy Wang Mertensmeyer 3.28
9 Victoria W. 3.48
10 Vanessa Yeo 3.52
11 Brandon Ly 2.70
12 Violet Hill 3.24
13 Ailun S. 1.92
14 Lin ZHANG 2.30
15 Bryant Mao 1.87
16 Yue Yuan 3.51
17 Rui Xu 4.15
18 Lisa Xue 2.11
19 Kiko Zang 2.12
20 Fiona Pay 3.97
21 Daniel Huang 4.14
22 Michelle Shao 2.10
23 Federico Duarte 3.86
24 Chuli Duan 4.39
25 Susie WANG 3.56
26 Brian (錢源偉) Chien 2.93
27 Yao Chen 3.69
28 Jasper Yang 3.34
29 Weiyi Li 李玮仪 3.57
30 Eileen Wu 2.90
31 Mark Logan MP 3.42
32 孙莹 Daisy 1.83
33 Yunna ( Tea ) Qiu 4.11
34 Hang Io 3.85
35 Evan Zeng 3.60
36 Louis Liu 4.18
37 Fangting Gu 3.03
38 Rita Yang 3.32
39 Alex Xie 3.91
40 Hao-Ting Chang 3.64
41 Simin Pan 4.36
42 Maggie Peng 2.01
43 Jaoui Liu 2.55
44 Siqi Wang 4.07
45 Felix Cheng 3.01
46 Amy Ly 2.30
47 Winston Chow 1.67
48 Sheng-Hung Lee 4.22
49 Momo Estrella 3.08


# Saving

True

In [132]:
df_basic_info = pd.DataFrame(basic_info_data_all, 
                             columns=['URL', 'Name', 'Title', 'Location'])
df_contacts = pd.DataFrame(contacts_data_all,
                           columns=['URL', 'Name', 'Contact'])
df_experience = pd.DataFrame(experience_data_all,
                             columns=['URL', 'Name', 'Company', 'Title', 
                                      'Start Date', 'End Date'])
df_education = pd.DataFrame(education_data_all,
                            columns=['URL', 'Name', 'Institution', 
                                     'Degree', 'Field of Study', 
                                     'Start Year', 'End Year'])
df_language = pd.DataFrame(language_data_all,
                           columns=['URL', 'Name', 'Language'])

In [136]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('../result/RUI.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet. you could write different string like above if you want
df_basic_info.to_excel(writer, sheet_name='Basic Info')
df_contacts.to_excel(writer, sheet_name='Contacts')
df_experience.to_excel(writer, sheet_name='Experience')
df_education.to_excel(writer, sheet_name='Education')
df_language.to_excel(writer, sheet_name='Language')

# Close the Pandas Excel writer and output the Excel file.
writer.save()
writer.close()

/Users/alee/.pyenv/versions/3.8.1/envs/linkedin/lib/python3.8/site-packages/xlsxwriter/workbook.py:329: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
